# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import glob 
import os 
import math
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter, MonthLocator


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("Output/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,8.33,90,50,12.90,TF,1708389022
1,1,kochubey,44.3861,46.5825,1.47,82,100,4.29,RU,1708389022
2,2,isafjordur,66.0755,-23.1240,1.94,97,100,3.94,IS,1708389022
3,3,invercargill,-46.4000,168.3500,13.90,57,48,5.81,NZ,1708388961
4,4,vilyuchinsk,52.9306,158.4028,-6.85,91,100,1.99,RU,1708389024
5,5,tazovsky,67.4667,78.7000,-11.68,91,100,9.17,RU,1708389024
6,6,waitangi,-43.9535,-176.5597,16.01,64,10,6.26,NZ,1708389025
7,7,camana,-16.6228,-72.7111,23.80,86,100,1.99,PE,1708389026
8,8,darab,28.7519,54.5444,8.15,42,0,1.19,IR,1708389027
9,9,amuntai,-2.4177,115.2494,29.32,70,6,1.79,ID,1708389025


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

pip install geoviews

In [7]:


# Configure the map plot
from matplotlib.colors import LinearSegmentedColormap
colors  = ['green', 'darkorange', 'firebrick'] 
cm      = LinearSegmentedColormap.from_list("Custom", colors, N=100)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50
plt.figure(figsize=(100,100))
m3 = city_data_df.hvplot.points(x='Lng', y='Lat', geo = True, tiles = "OSM", marker='circle', 
                                size=city_data_df["Humidity"], frame_height=1000, fontsize=18, 
                                xlabel="Longitude", ylabel="Latitude", alpha=10000000.0, cmap=cm, color=np.random.randint(0,579,579),
                                fill_alpha=15, label="City")
m3

:Overlay
   .Tiles.I     :Tiles   [x,y]
   .Points.City :Points   [Lng,Lat]   (_color,_size)

<Figure size 10000x10000 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] < 35) & (city_data_df['Wind Speed'] < 2.5)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,camana,-16.6228,-72.7111,23.80,86,100,1.99,PE,1708389026
9,9,amuntai,-2.4177,115.2494,29.32,70,6,1.79,ID,1708389025
12,12,kindu,-2.9500,25.9500,21.93,94,100,1.25,CD,1708389029
28,28,blackmans bay,-43.0167,147.3167,21.40,60,100,1.77,AU,1708389036
30,30,freetown,8.4840,-13.2299,26.19,90,58,2.49,SL,1708389037
...,...,...,...,...,...,...,...,...,...,...
558,558,owando,-0.4819,15.8999,24.66,77,100,1.57,CG,1708389281
559,559,katabu,-4.9333,122.5167,28.42,80,42,2.44,ID,1708389281
560,560,pizarro,4.9533,-77.3660,26.17,91,100,1.86,CO,1708389281
566,566,andalgala,-27.5819,-66.3165,22.18,62,0,2.48,AR,1708389284


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  city_data_df[['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
7,7,camana,PE,-16.6228,-72.7111,86,
9,9,amuntai,ID,-2.4177,115.2494,70,
12,12,kindu,CD,-2.9500,25.9500,94,
28,28,blackmans bay,AU,-43.0167,147.3167,60,
30,30,freetown,SL,8.4840,-13.2299,90,
...,...,...,...,...,...,...,...
558,558,owando,CG,-0.4819,15.8999,77,
559,559,katabu,ID,-4.9333,122.5167,80,
560,560,pizarro,CO,4.9533,-77.3660,91,
566,566,andalgala,AR,-27.5819,-66.3165,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
#from config import geoapify_key
#geoapify_key = "eb43efe87ad84d32a9e4cb0f86666dfc"

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"]  = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
camana - nearest hotel: Caja Arequipa
amuntai - nearest hotel: Grand Balqis Hotel
kindu - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
freetown - nearest hotel: Formal Leona hotel
ouesso - nearest hotel: Auberge Nianina
ambon - nearest hotel: Hotel Hero
monteagudo - nearest hotel: Residencial San José
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
seven pagodas - nearest hotel: Radisson Temple Bay
kimbe - nearest hotel: Kimbe Bay Hotel
port alfred - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
sola - nearest hotel: No hotel found
karachi - nearest hotel: Quetta Agha Hotel
queenstown - nearest hotel: Queens Hotel
ribeira grande - nearest hotel: Hospedaria JSF
sinabang - nearest hotel: No hotel found
newnham - nearest hotel: Riverside Hotel Motel
bambari - nearest hotel: No hotel found
hedaru - nearest hotel: Maharage Palace
brisas barra de suchiate - nearest hotel: No hotel found
mermoz boaba

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
7,7,camana,PE,-16.6228,-72.7111,86,Caja Arequipa
9,9,amuntai,ID,-2.4177,115.2494,70,Grand Balqis Hotel
12,12,kindu,CD,-2.9500,25.9500,94,No hotel found
28,28,blackmans bay,AU,-43.0167,147.3167,60,Villa Howden
30,30,freetown,SL,8.4840,-13.2299,90,Formal Leona hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

from matplotlib.colors import LinearSegmentedColormap
colors  = ['green', 'darkorange', 'firebrick'] 
cm      = LinearSegmentedColormap.from_list("Custom", colors, N=100)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50
plt.figure(figsize=(100,100))
m4 = hotel_df.hvplot.points(x='Lng', y='Lat', geo=True, tiles = 'OSM', frame_width = 700, 
                            frame_height = 500, color='City', alpha=0.6, hover=True,
                            hover_cols='all', use_index=False, s='Humidity',
                            scale= 1.2)
m4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,City_ID,Country,Hotel Name)

<Figure size 10000x10000 with 0 Axes>